<img src="images/nalu.png" align=right width=40%>
# Neural Arithmetic Logic Units
Author: Jin Yeom (jinyeom@utexas.edu)

## Contents
- [Neural accumulator](#Neural-accumulator)

In [1]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torchsummary import summary
from tqdm import tqdm_notebook as tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device =", device)

device = cuda


## Neural accumulator

In [4]:
class NAC(nn.Module):
    def __init__(self, in_features, out_features):
        super(NAC, self).__init__()
        self.W_hat = nn.Linear(in_features, out_features)
        self.M_hat = nn.Linear(in_features, out_features)
        
    def forward(self, x):
        W = F.tanh(self.)

SyntaxError: invalid syntax (<ipython-input-4-8db9cf8644e6>, line 2)